In [3]:
import sys
sys.path.append('..')

import os
project_root = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))

TRAIN_VERSION="v3"
data_path = "../data/data_1h_2023.csv"

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
import warnings

warnings.filterwarnings("ignore")

import sys

sys.path.append("..")
from envs.trading_env import MyTradingEnv


class ConsistentAgentComparator:
    def discretize_state(self, observation):
        return tuple(map(int, observation))

    def run(self):
        print("СРАВНЕНИЕ АГЕНТОВ")
        print("=" * 50)

        agents = {
            "Q-Learning": f"../training_data/checkpoints/exp_qlearn_{TRAIN_VERSION}/final_agent.pkl",
            # "SARSA": f"../training_data/checkpoints/exp_sarsa_{TRAIN_VERSION}/final_agent.pkl",
            # "SARSA-λ": f"../training_data/checkpoints/exp_sarsa_lambda_{TRAIN_VERSION}/final_agent.pkl",
            # "Monte Carlo": f"../training_data/checkpoints/exp_monte_carlo_{TRAIN_VERSION}/final_agent.pkl",
        }

        df_full = pd.read_csv(data_path, index_col=0, parse_dates=True)
        df = df_full.copy()

        results = []

        for name, path in agents.items():
            print(f"\n{name}:")

            if not Path(path).exists():
                print(f"  Файл не найден")
                continue

            try:
                with open(path, "rb") as f:
                    agent_data = pickle.load(f)

                class TestAgent:
                    def __init__(self, data, discretize_func):
                        self.q_table = data.get("q_table", {})
                        self.discretize = discretize_func

                    def select_action(self, state, greedy=True):
                        state_key = self.discretize(state)
                        if state_key in self.q_table:
                            q_vals = self.q_table[state_key]
                            if (
                                isinstance(q_vals, (list, np.ndarray))
                                and len(q_vals) == 3
                            ):
                                return np.argmax(q_vals)
                        return 1

                agent = TestAgent(agent_data, self.discretize_state)

                env = MyTradingEnv(
                    df=df.copy(),
                    initial_balance=1000,
                    window_size=10,
                    commission=0.0001,
                    slippage=0.0005,
                    max_holding_time=60 * 24,
                    max_drawdown_threshold=0.05,
                    holding_threshold=24,
                )

                state, info = env.reset()
                done = False

                while not done:
                    action = agent.select_action(state, greedy=True)
                    next_state, reward, done, truncated, info = env.step(action)
                    state = next_state

                metrics = env.get_metrics()

                env.reset()
                portfolio_values = []
                state, info = env.reset()
                done = False

                while not done:
                    action = agent.select_action(state, greedy=True)
                    next_state, reward, done, truncated, info = env.step(action)
                    portfolio_values.append(info["portfolio_value"])
                    state = next_state

                portfolio_values = np.array(portfolio_values)
                peak = np.maximum.accumulate(portfolio_values)
                drawdowns = (peak - portfolio_values) / peak
                max_portfolio_drawdown = np.max(drawdowns) if len(drawdowns) > 0 else 0

                results.append(
                    {
                        "Агент": name,
                        "Доходность %": (env._portfolio_value / 1000 - 1) * 100,
                        "Конечный баланс": env._portfolio_value,
                        "Сделок": metrics.get("total_trades", 0),
                        "Win Rate %": metrics.get("win_rate", 0),
                        "Средний PnL": metrics.get("avg_pnl", 0),
                        "Max Drawdown сделок": metrics.get("max_drawdown", 0) * 100,
                        "Max Drawdown портфеля": max_portfolio_drawdown * 100,
                        "Ср. время удержания": metrics.get("avg_holding_time", 0),
                        "Закрыто по просадке": metrics.get(
                            "trades_closed_by_drawdown", 0
                        ),
                        "Закрыто по времени": metrics.get("trades_closed_by_time", 0),
                    }
                )

                print(f"  Доходность: {(env._portfolio_value / 1000 - 1) * 100:+.2f}%")
                print(f"  Сделок: {metrics.get('total_trades', 0)}")
                print(
                    f"  Макс. просадка сделок: {metrics.get('max_drawdown', 0) * 100:.2f}%"
                )
                print(f"  Макс. просадка портфеля: {max_portfolio_drawdown * 100:.2f}%")

            except Exception as e:
                print(f"  Ошибка: {e}")
                import traceback

                traceback.print_exc()

        if results:
            print("\n" + "=" * 50)
            print("ИТОГИ:")
            print("=" * 50)

            df_results = pd.DataFrame(results)

            print("\nСортировка по доходности:")
            print(
                df_results.sort_values("Доходность %", ascending=False)[
                    [
                        "Агент",
                        "Доходность %",
                        "Конечный баланс",
                        "Max Drawdown портфеля",
                        "Сделок",
                    ]
                ].to_string(index=False)
            )

            print("\n\nСортировка по минимальной просадке портфеля:")
            print(
                df_results.sort_values("Max Drawdown портфеля", ascending=True)[
                    ["Агент", "Max Drawdown портфеля", "Доходность %", "Сделок"]
                ].to_string(index=False)
            )

            print("\n\nСортировка по максимальному балансу:")
            print(
                df_results.sort_values("Конечный баланс", ascending=False)[
                    [
                        "Агент",
                        "Конечный баланс",
                        "Доходность %",
                        "Max Drawdown портфеля",
                    ]
                ].to_string(index=False)
            )

            print("\n\nПолная таблица результатов:")
            print(df_results.to_string(index=False))

            df_results.to_csv("consistent_comparison_results.csv", index=False)
            print(f"\nРезультаты сохранены в consistent_comparison_results.csv")

            print("\n" + "=" * 50)
            print("АНАЛИЗ ЛУЧШЕГО АГЕНТА:")
            print("=" * 50)

            df_results["Коэфф. доходность/просадка"] = df_results["Доходность %"] / (
                df_results["Max Drawdown портфеля"] + 0.01
            )
            best_agent_idx = df_results["Коэфф. доходность/просадка"].idxmax()
            best_agent = df_results.loc[best_agent_idx]

            print(
                f"Лучший агент по соотношению доходность/просадка: {best_agent['Агент']}"
            )
            print(f"Доходность: {best_agent['Доходность %']:.2f}%")
            print(
                f"Макс. просадка портфеля: {best_agent['Max Drawdown портфеля']:.2f}%"
            )
            print(f"Соотношение: {best_agent['Коэфф. доходность/просадка']:.2f}")

        else:
            print("\nНет результатов для сравнения")


if __name__ == "__main__":
    comparator = ConsistentAgentComparator()
    comparator.run()

СРАВНЕНИЕ АГЕНТОВ

Q-Learning:
  Доходность: +18.29%
  Сделок: 202
  Макс. просадка сделок: 3.31%
  Макс. просадка портфеля: 23.94%

ИТОГИ:

Сортировка по доходности:
     Агент  Доходность %  Конечный баланс  Max Drawdown портфеля  Сделок
Q-Learning     18.293338      1182.933376              23.943251     202


Сортировка по минимальной просадке портфеля:
     Агент  Max Drawdown портфеля  Доходность %  Сделок
Q-Learning              23.943251     18.293338     202


Сортировка по максимальному балансу:
     Агент  Конечный баланс  Доходность %  Max Drawdown портфеля
Q-Learning      1182.933376     18.293338              23.943251


Полная таблица результатов:
     Агент  Доходность %  Конечный баланс  Сделок  Win Rate %  Средний PnL  Max Drawdown сделок  Max Drawdown портфеля  Ср. время удержания  Закрыто по просадке  Закрыто по времени
Q-Learning     18.293338      1182.933376     202   28.712871     1.020807             3.309306              23.943251             5.242574         